In [1]:
"""連続型確率変数"""
#離散型確率変数と連続型確率変数の主な違いは、取り得る値が離散的か連続的かという点である。連続型確率変数の場合、計算過程に積分が含まれるなど離散型に比べると少し数式がややこしくなる。

#NumpyとMatplotlibをインポート
import numpy as np
import matplotlib.pyplot as plt

%precision 3
%matplotlib inline

#積分はSciPyのintegralモジュールを用いて実装していく。integralの計算に、結果に問題はないもののwarningが出る処理が含まれるため、あらかじめwarningを抑制しておく
from scipy import integrate
import warnings

#積分に関するwarningを出力しないようにする。
warnings.filterwarnings('ignore', category=integrate.IntegrationWarning)